In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import ipyvolume as ipv

In [ ]:
def saturate(x):
    return np.clip(x, 0, 1)

def horizon_clipping_sphere_approx(sinSigma, cosOmega):
    l = sinSigma*sinSigma
    Fz = cosOmega * l 
    sum = (l * l + Fz) / (l + 1)
    return sum

def horizon_clipping_sphere(sinSigma, cosOmega):
    squaredSinSigma = sinSigma * sinSigma
    sinSigma = np.sqrt(squaredSinSigma)
    cosSigma = np.sqrt(1 - squaredSinSigma)
    sigma = np.arcsin(sinSigma)

    sinOmega = np.sqrt(1 - cosOmega * cosOmega)
    omega = np.arccos(cosOmega)

    sinGamma = saturate(cosSigma / sinOmega)
    squaredCosGamma = 1 - sinGamma * sinGamma
    cosGamma = np.sqrt(squaredCosGamma)
    gamma = np.arcsin(sinGamma)

    sum = 0
    term0 = np.pi * cosOmega * squaredSinSigma
    G = -2 * sinOmega * cosSigma * cosGamma + np.pi/2 - gamma + sinGamma * cosGamma
    H = cosOmega * (cosGamma * np.sqrt(saturate(squaredSinSigma - squaredCosGamma)) + squaredSinSigma * np.arcsin(cosGamma / sinSigma))
    if ((0 <= omega) and (omega < np.pi/2 - sigma)):
        sum = term0
    elif ((np.pi/2 - sigma <= omega) and (omega < np.pi/2)):
        sum = term0 + G - H
    elif ((np.pi/2 <= omega) and (omega < np.pi/2 + sigma)):
        sum = G + H

    sum = sum / np.pi
    return sum


horizon_clipping_sphere_vec = np.vectorize(horizon_clipping_sphere)
sinSigma = np.arange(0.0001, 1, 0.01)
cosOmega = np.arange(0.0001, 1, 0.01)
sinSigma, cosOmega = np.meshgrid(sinSigma, cosOmega)
lut = horizon_clipping_sphere_vec(sinSigma, cosOmega)
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# surf = ax.plot_surface(sinSigma, cosOmega, lut, cmap=cm.coolwarm)
# plt.show()

ipv.clear()
ipv.plot_surface(sinSigma, cosOmega, lut)
ipv.show()


In [ ]:
approx = horizon_clipping_sphere_approx(sinSigma, cosOmega)
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# surf = ax.plot_surface(sinSigma, cosOmega, lut, cmap=cm.coolwarm)
# plt.show()

ipv.plot_surface(sinSigma, cosOmega, approx)
ipv.show()
